In [43]:
import pandas as pd
import numpy as np
import mysql.connector

In [2]:
# !pip install mysql-connector-python

In [3]:
# Set up a MySQL connection
cnx = mysql.connector.connect(user='root', password='my123sql',
                              host='localhost', database='intest')

In [4]:
query_user = "SELECT * FROM users"
df_user = pd.read_sql(query_user, cnx)
if df_user.size>0:
    print("UserDB:"+str(df_user.shape))

query_test = "SELECT * FROM test_db"
df_test= pd.read_sql(query_test, cnx)
if df_test.size>0:
    print("TestDB:"+str(df_test.shape))

query_questions = "SELECT * FROM questions"
df_questions= pd.read_sql(query_questions, cnx)
if df_questions.size>0:
    print("QuestionsDB:"+str(df_questions.shape))

query_options = "SELECT * FROM mcq_options"
df_options= pd.read_sql(query_options, cnx)
if df_options.size>0:
    print("OptionsDB:"+str(df_options.shape))

query_responses = "SELECT * FROM responses"
df_responses= pd.read_sql(query_responses, cnx)
if df_responses.size>0:
    print("ResponsesDB:"+str(df_responses.shape))

query_olap_test = "SELECT * FROM olap_test"
df_olap_test= pd.read_sql(query_olap_test, cnx)
# if df_olap_test.size>0:
print("Olap_testDB:"+str(df_olap_test.shape))
    
query_olap_question = "SELECT * FROM olap_question"
df_olap_question= pd.read_sql(query_olap_question, cnx)
# if df_olap_question.size>0:
print("Olap_questionDB:"+str(df_olap_question.shape))

UserDB:(5, 7)
TestDB:(1, 9)
QuestionsDB:(3, 7)
OptionsDB:(3, 6)
ResponsesDB:(12, 6)
Olap_testDB:(0, 10)
Olap_questionDB:(0, 7)


In [5]:
class dataframes:
    user=test=questions=options=responses=olap_test=olap_question=pd.DataFrame()
    def __init__(self):
        cnx = mysql.connector.connect(user='root', password='my123sql',
                              host='localhost', database='intest')
        
        query_user = "SELECT * FROM users"
        self.user = pd.read_sql(query_user, cnx)
        
        query_test = "SELECT * FROM test_db"
        self.test= pd.read_sql(query_test, cnx)
        
        query_questions = "SELECT * FROM questions"
        self.questions= pd.read_sql(query_questions, cnx)
        
        query_options = "SELECT * FROM mcq_options"
        self.options= pd.read_sql(query_options, cnx)
        
        query_responses = "SELECT * FROM responses"
        self.responses= pd.read_sql(query_responses, cnx)
        
        query_olap_test = "SELECT * FROM olap_test"
        self.olap_test= pd.read_sql(query_olap_test, cnx)
        
        query_olap_question = "SELECT * FROM olap_question"
        self.olap_question= pd.read_sql(query_olap_question, cnx)
        
        cnx.close()

In [6]:
df=dataframes()
df.user

,email,firstName,lastName,password,role,profilePicUrl,id
0,amitabh@gmail.com,Amitabh,Bachchan,password,teacher,https://upload.wikimedia.org/wikipedia/commons...,1
1,shahrukh@gmail.com,Shahrukh,Khan,password,student,https://upload.wikimedia.org/wikipedia/commons...,2
2,salman@gmail.com,Salman,Khan,password,student,https://upload.wikimedia.org/wikipedia/commons...,3
3,aamir@gmail.com,Aamir,Khan,password,student,https://upload.wikimedia.org/wikipedia/commons...,4
4,hrithik@gmail.com,Hrithik,Roshan,password,student,https://upload.wikimedia.org/wikipedia/commons...,5


In [34]:
import json
def checkAnswers(df: dataframes, testID: int):
    def getOptionId(questionId):
        question=df.questions.query('id==@questionId')
        optionId=question.optionsId.iloc[0]
        return optionId
    
    def getMarks(questionId):
        question=df.questions.query('id==@questionId')
        marks=question.maxMarks.iloc[0]
        return marks
    
    def getCorrectAnswer(optionId):    
        option=df.options.query('id==@optionId')
        correctOption=option.correctOption.iloc[0]
        optionNumber="option"+str(correctOption)
        correctAnswer=option[optionNumber].iloc[0]
        return correctAnswer
    
    responseDF=df.responses.query('testId==@testID')
    questionsIdList=responseDF.questionId.unique()
    answer={}
    marks={}
    if len(questionsIdList)==0:
        return {"error":"no questions exist in selected test"}
    
    for questionId in questionsIdList:
        optionId=getOptionId(questionId)
        correctAnswer=getCorrectAnswer(optionId)
        answer[questionId]=correctAnswer
        marks[questionId]=getMarks(questionId)
    
    for index,response in df.responses.iterrows():
        body=response['body']
    
    print(answer)
    display()
    
    
    

In [ ]:
testId=1
responseDF=df.responses.query('testId==@testID')
for index, row in responseDF:
    print(index)

In [45]:
for index, row in df.responses.iterrows():
    row['obtainedMarks']=0
    print(index,row["testId"], row["body"],row['obtainedMarks'])
display(df.responses)

0 1 New Delhi 0
1 1 Narendra Modi 0
2 1 Tiger 0
3 1 Mumbai 0
4 1 Rahul Gandhi 0
5 1 Lion 0
6 1 Mumbai 0
7 1 Narendra Modi 0
8 1 Lion 0
9 1 New Delhi 0
10 1 Rahul Gandhi 0
11 1 Tiger 0


,id,questionId,userId,testId,body,obtainedMarks
0,1,1,2,1,New Delhi,5
1,2,2,2,1,Narendra Modi,5
2,3,3,2,1,Tiger,5
3,4,1,3,1,Mumbai,0
4,5,2,3,1,Rahul Gandhi,0
5,6,3,3,1,Lion,0
6,13,1,4,1,Mumbai,0
7,14,2,4,1,Narendra Modi,5
8,15,3,4,1,Lion,0
9,16,1,5,1,New Delhi,5


In [35]:
checkAnswers(df,1)

{1: 'New Delhi', 2: 'Narendra Modi', 3: 'Tiger'}


In [ ]:
# Close the MySQL connection
# cnx.close()

In [ ]:
df.responses.query('testId==1')